# Imports

In [1]:
# Standard library imports
import time
import random

# Third party imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch import load
from lmfit import Model

# Load data and split in train/test

In [2]:
X = load("outputs/X_noisy.pt")
y = load("outputs/y_noisy.pt")

# split in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(f'X_train.shape = {X_train.shape} ; X_test.shape = {X_test.shape}')
print(f'y_train.shape = {y_train.shape} ; y_test.shape = {y_test.shape}')

X_train.shape = torch.Size([382433, 31]) ; X_test.shape = torch.Size([42493, 31])
y_train.shape = torch.Size([382433, 3]) ; y_test.shape = torch.Size([42493, 3])


# Define parameters and functions

In [3]:
def wasabi(x, dB0, B1, c, d, freq, tp):
    """WASABI function for simultaneous determination of B1 and B0. For more information read:
    Schuenke et al. Magnetic Resonance in Medicine, 77(2), 571–580. https://doi.org/10.1002/mrm.26133"""
    return np.abs(c - d * np.square(np.sin(np.arctan((B1 / (freq / gamma)) / (x - dB0)))) *
                  np.square(np.sin(np.sqrt(np.square(B1 / (freq / gamma)) +
                                           np.square(x - dB0)) * freq * 2 * np.pi * tp / 2)))

B0 = 3
B1_nom = 1.25*B0  # in µT here... so no 1e-6 factor needed
tp = 0.015/B0
gamma = 42.577
offsets = np.linspace(-2, 2, 31)

# Fit data with analytic approximation

In [4]:
model = Model(wasabi)
params = model.make_params()
params['freq'].set(gamma * B0, vary=False)
params['tp'].set(tp, vary=False)
params['dB0'].set(0)
params['B1'].set(B1_nom, min=0)
params['c'].set(0.9)
params['d'].set(1.4, min=0)

dw_list = []
dw_diff = []
b1_list = []
b1_diff = []

count = 0
n_total = X_test.shape[0]
loopstart = time.time()
for i in range(X_test.shape[0]):
    data = X_test[i,:].numpy()
    y_ = y_test[i].numpy()
    
    # set parameters within +/- 10% of true value to increase fit rebustness. 
    # Of course this can NOT be done in real measurements later, because we don't know the true value in advance
    params['dB0'].set(y_[0]*random.uniform(0.9, 1.1))
    params['B1'].set(y_[2]*B1_nom*random.uniform(0.9, 1.1), min=0)
    
    # perform fitting
    res_ = model.fit(np.abs(data), params, x=offsets, max_nfev=100)
    vals = res_.best_values
    
    # write values
    dw_list.append(vals['dB0'])
    dw_diff.append(vals['dB0']-y_[0])
    b1_list.append(vals['B1'])
    b1_diff.append(vals['B1']-y_[2]*B1_nom)

    # update progress bar and estimated time
    b = int(50 * count / n_total)
    left = int(50 - b)
    count += 1
    loopremain = (time.time() - loopstart) * (n_total - count) / (count * 50)
    print('[' + '#' * b + '-' * left + ']' +
          f' Estimated remaining time {loopremain:.1f} minutes.', end='\r')

[--------------------------------------------------] Estimated remaining time 5.5 minutes..

<ipython-input-3-8fb92ecf0f17>:4: RuntimeWarning: divide by zero encountered in true_divide
  return np.abs(c - d * np.square(np.sin(np.arctan((B1 / (freq / gamma)) / (x - dB0)))) *


# Evaluate results

In [ ]:
print(f'Mean dB0 error = {np.mean(np.abs(dw_diff)):.3f} ppm')
print(f'Max dB0 error = {np.max(np.abs(dw_diff)):.3f} ppm')
print(' ')
print(f'Mean B1 error = {np.mean(np.abs(b1_diff)):.3f} µT')
print(f'Max B1 error = {np.max(np.abs(b1_diff)):.3f} µT')